# In-class transcript from Lecture 1, January 7, 2020


# Imports and defs for lecture

In [1]:
# These are the standard imports for CS 111. 
# This list may change as the quarter goes on.

import os
import math
import time
import struct
import json
import pandas as pd
import networkx as nx
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
from scipy import integrate
import matplotlib
# %matplotlib inline
%matplotlib tk
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d



<c><h2>The temperature problem:</h2></c>

A cabin in the snow has all its walls at 0 degrees Celsius,
except for a radiator at 100 degrees Celsius on one wall.
What is the temperature at each point inside the cabin?

For this model, we take the cabin to be 2-dimensional and square, and we discretize the interior
of the cabin into a grid of k by k points. The temperature at any given point is (approximately)
the average of the temperatures at the four neighboring points. 
This fact gives us one linear equation at each point, for a total of ndim = k<sup>2</sup> linear equations
in the k<sup>2</sup> variables representing the temperatures at each point.

We express this system of ndim linear equations in matrix form as A * t = b, where A is an ndim-by-ndim
matrix (most of whose entries are zero), b is a right-hand size vector of ndim entries that among other
things encodes the boundary temperatures, and the unknown vector t is the temperature at each interior point.

This code sets up the matrix A and the vector b, uses scipy to solve the linear system for t, and 
uses matplotlib to visualize t.

In [2]:
# Making matrices for the temperature problem (i.e. the discrete Laplacian operator)

# Two dimensions 
def make_A_2D(k):
    """Create the matrix of the discrete Laplacian operator in two dimensions on a k-by-k grid.
    Parameters: 
      k: number of grid points in each dimension.
    Outputs:
      A: the sparse k**2-by-k**2 matrix representing the finite difference approximation to Laplace's equation.
    """
    # First make a list with one triple (row, column, value) for each nonzero element of A
    triples = []
    for x in range(k):
        for y in range(k):
                
            # what row of the matrix is grid point (x,y)?
            row = x + k*y

            # the diagonal element in this row
            col = row
            triples.append((row, col, 4.0))
            # connect to grid neighbors in x dimension
            if x > 0:
                col = row - 1
                triples.append((row, col, -1.0))
            if x < k - 1:
                col = row + 1
                triples.append((row, col, -1.0))
            # connect to grid neighbors in y dimension
            if y > 0:
                col = row - k
                triples.append((row, col, -1.0))
            if y < k - 1:
                col = row + k
                triples.append((row, col, -1.0))

    # Finally convert the list of triples to a scipy sparse matrix
    ndim = k*k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values, (rownum, colnum)), shape = (ndim, ndim))
    
    return A 


In [4]:
# Make right-hand side vector for the 2D Poisson equation 

def make_b(k, top = 0, bottom = 0, left = 0, right = 0):
    """Create the right-hand side for the temperature problem on a k-by-k grid.
    Parameters: 
      k: number of grid points in each dimension.
      top: list of k values for top boundary (optional, defaults to 0)
      bottom: list of k values for bottom boundary (optional, defaults to 0)
      left: list of k values for top boundary (optional, defaults to 0)
      right: list of k values for top boundary (optional, defaults to 0)
    Outputs:
      b: the k**2 element vector (as a numpy array) for the rhs of the Poisson equation with given boundary conditions
    """
    # Start with a vector of zeros
    ndim = k*k
    b = np.zeros(shape = ndim)
    
    # Fill in the four boundaries as appropriate
    b[0        : k       ] += top
    b[ndim - k : ndim    ] += bottom
    b[0        : ndim : k] += left
    b[k-1      : ndim : k] += right
    
    return b
    
    
def radiator(k, width = .3, temperature = 100.):
    """Create one wall with a radiator
    Parameters: 
      k: number of grid points in each dimension; length of the wall.
      width: width of the radiator as a fraction of length of the wall (defaults to 0.2)
      temperature: temperature of the radiator (defaults to 100)
    Outputs:
      wall: the k element vector (as a numpy array) for the boundary conditions at the wall
    """
    rad_start = int(k * (0.5 - width/2))
    rad_end = int(k * (0.5 + width/2))
    wall = np.zeros(k)
    wall[rad_start : rad_end] = temperature
    
    return wall


# Lecture starts here

In [5]:
# Poisson equation (temperature equation) in 2D.

# Choose how finely to discretize
k = 100

# Get the matrix (the finite-difference discretization of the Laplace operator)
A = make_A_2D(k)

# Get the right-hand side, with boundary conditions
rad_wall = radiator(k)
b = make_b(k, top = rad_wall)


In [6]:
A

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 49600 stored elements in Compressed Sparse Row format>

In [7]:
b

array([0., 0., 0., ..., 0., 0., 0.])

In [8]:
b[:50]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100., 100., 100., 100., 100., 100.])

In [9]:
x = spla.spsolve(A,b)


In [10]:
x

array([0.03453673, 0.0692933 , 0.10449331, ..., 0.01420413, 0.00947631,
       0.00474022])

In [11]:
T = x.reshape(k, k)
plt.figure()
plt.imshow(T, cmap=cm.hot)
plt.xlabel('x0')
plt.ylabel('x1')
plt.title('2-dimensional Poisson equation (temperature)')

Text(0.5,1,'2-dimensional Poisson equation (temperature)')

In [12]:
X, Y = np.meshgrid(range(k), range(k))
%matplotlib tk
fig = plt.figure()
ax = fig.gca(projection='3d')
ax = fig.gca()
ax.plot_surface(X, Y, T, cmap=cm.hot)